# QROM

In [ ]:
import cirq
import numpy as np
import cirq_qubitization
import cirq_qubitization.cirq_infra.testing as cq_testing
from cirq_qubitization.jupyter_tools import display_gate_and_compilation, show_bloq
from typing import *

## `QROM`
Gate to load data[l] in the target register when the selection register stores integer l.

In [ ]:
from cirq_qubitization.cirq_algos import QROM

qrom = QROM([1, 2, 3, 4, 5])
g = cq_testing.GateHelper(
    qrom
)
cirq.Circuit(cirq.decompose_once(g.operation))

display_gate_and_compilation(g)

In [ ]:
from cirq_qubitization.bit_tools import iter_bits
data = [[1, 2, 3, 4, 5]]
for selection_integer in range(qrom.iteration_length):
    qubit_vals = {x: 0 for x in g.all_qubits}
    qubit_vals |= zip(
        g.quregs['selection'], iter_bits(selection_integer, g.r['selection'].bitsize)
    )

    initial_state = [qubit_vals[x] for x in g.all_qubits]
    for ti, d in enumerate(data):
        target = g.quregs[f"target{ti}"]
        qubit_vals |= zip(target, iter_bits(d[selection_integer], len(target)))
    final_state = [qubit_vals[x] for x in g.all_qubits]
    cq_testing.assert_circuit_inp_out_cirqsim(
        g.decomposed_circuit, g.all_qubits, initial_state, final_state
    )

We can also use QROM to load floating point data, typically by converting to a fixed point representation. Complex numbers can similarly be loaded by splitting the data array into real and complex values.

In [ ]:

import select
from cirq_qubitization.bit_tools import fixed_point_bits_to_float, float_to_fixed_point_bits 
data = [10.38, -2.15, -0.999, 12.33, -19.34]
bin_width = 10
frac_width = 4
data_fixed_point = [int(float_to_fixed_point_bits(d, bin_width, frac_width), 2) for d in data]
qrom = QROM(data_fixed_point)
g = cq_testing.GateHelper(
    qrom
)
cirq.Circuit(cirq.decompose_once(g.operation))

for selection_integer in range(qrom.iteration_length):
    qubit_vals = {x: 0 for x in g.all_qubits}
    qubit_vals |= zip(
        g.quregs['selection'], iter_bits(selection_integer, g.r['selection'].bitsize)
    )

    initial_state = [qubit_vals[x] for x in g.all_qubits]
    for ti, d in enumerate([data_fixed_point]):
        target = g.quregs[f"target{ti}"]
        qubit_vals |= zip(target, iter_bits(data_fixed_point[selection_integer], len(target)))
    final_state = [qubit_vals[x] for x in g.all_qubits]
    cq_testing.assert_circuit_inp_out_cirqsim(
        g.decomposed_circuit, g.all_qubits, initial_state, final_state
    )

    target_val = ''.join(f"{tv}" for tv in [final_state[g.all_qubits.index(q)] for q in g.quregs["target0"]])
    loaded_value = fixed_point_bits_to_float(target_val, frac_width)
    fp_int = float_to_fixed_point_bits(data[selection_integer], bin_width, frac_width)
    expected_value = fixed_point_bits_to_float(fp_int, frac_width)
    print("loaded = {}, expected = {}, exact = {}".format(loaded_value, expected_value, data[selection_integer]))